In [1]:
# !pip install albumentations opencv-python --quiet

import os
import cv2
import albumentations as A

# Шляхи
input_dir = "/kaggle/input/initial-yolo-cats/cats-yolo-dataset"
output_dir = "/kaggle/working/cats-yolo-dataset-aug"
num_aug_per_image = 7

# Функції перетворення форматів bbox
def yolo_to_voc(bbox, img_w, img_h):
    x, y, w, h = bbox
    x_center = x * img_w
    y_center = y * img_h
    w *= img_w
    h *= img_h
    x_min = x_center - w / 2
    y_min = y_center - h / 2
    x_max = x_center + w / 2
    y_max = y_center + h / 2
    return [x_min, y_min, x_max, y_max]

def voc_to_yolo(bbox, img_w, img_h):
    x_min, y_min, x_max, y_max = bbox
    x_min = max(0.0, min(x_min, img_w - 1))
    y_min = max(0.0, min(y_min, img_h - 1))
    x_max = max(0.0, min(x_max, img_w - 1))
    y_max = max(0.0, min(y_max, img_h - 1))
    x_center = (x_min + x_max) / 2 / img_w
    y_center = (y_min + y_max) / 2 / img_h
    w = (x_max - x_min) / img_w
    h = (y_max - y_min) / img_h
    return [
        max(0.0, min(1.0, x_center)),
        max(0.0, min(1.0, y_center)),
        max(0.0, min(1.0, w)),
        max(0.0, min(1.0, h)),
    ]

def clip_voc_bbox(bbox, img_w, img_h):
    x_min, y_min, x_max, y_max = bbox
    return [
        max(0.0, min(x_min, img_w - 1)),
        max(0.0, min(y_min, img_h - 1)),
        max(0.0, min(x_max, img_w - 1)),
        max(0.0, min(y_max, img_h - 1)),
    ]

# Створення директорій
for split in ["train", "valid"]:
    os.makedirs(f"{output_dir}/images/{split}", exist_ok=True)
    os.makedirs(f"{output_dir}/labels/{split}", exist_ok=True)

# Пайплайн аугментацій
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=10, p=0.3, border_mode=cv2.BORDER_CONSTANT),
    A.Blur(blur_limit=3, p=0.1),
    A.RandomScale(scale_limit=0.2, p=0.3),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05, p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.0, rotate_limit=0, p=0.3, border_mode=cv2.BORDER_CONSTANT)
], bbox_params=A.BboxParams(
    format='pascal_voc',
    label_fields=['class_labels'],
    min_visibility=0.01,
    check_each_transform=True
))

bad_boxes_total = 0
skipped_augs = 0

for split in ["train", "valid"]:
    img_dir = os.path.join(input_dir, split, "images")
    lbl_dir = os.path.join(input_dir, split, "labels")

    for fname in os.listdir(img_dir):
        if not fname.endswith(".jpg"):
            continue

        img_path = os.path.join(img_dir, fname)
        lbl_path = os.path.join(lbl_dir, fname.replace(".jpg", ".txt"))

        image = cv2.imread(img_path)
        if image is None or not os.path.exists(lbl_path):
            continue

        h, w = image.shape[:2]
        with open(lbl_path, "r") as f:
            lines = f.read().strip().split("\n")
        boxes_yolo = []
        class_labels = []
        for line in lines:
            if not line.strip():
                continue
            parts = line.strip().split()
            cls = int(parts[0])
            bbox = list(map(float, parts[1:]))
            if all(0.0 <= b <= 1.0 for b in bbox):
                boxes_yolo.append(bbox)
                class_labels.append(cls)
            else:
                bad_boxes_total += 1

        if not boxes_yolo:
            continue

        boxes_voc = [clip_voc_bbox(yolo_to_voc(b, w, h), w, h) for b in boxes_yolo]

        # Зберігаємо оригінал
        cv2.imwrite(os.path.join(output_dir, "images", split, fname), image)
        with open(os.path.join(output_dir, "labels", split, fname.replace(".jpg", ".txt")), "w") as f:
            for cls, b in zip(class_labels, boxes_yolo):
                f.write(f"{int(cls)} {' '.join(f'{v:.6f}' for v in b)}\n")

        # Аугментація
        for i in range(num_aug_per_image):
            try:
                augmented = transform(image=image, bboxes=boxes_voc, class_labels=class_labels)
                aug_img = augmented["image"]
                aug_boxes_voc = augmented["bboxes"]
                aug_labels = augmented["class_labels"]

                if not aug_boxes_voc:
                    skipped_augs += 1
                    continue

                aug_boxes_yolo = [voc_to_yolo(b, aug_img.shape[1], aug_img.shape[0]) for b in aug_boxes_voc]
                aug_name = fname.replace(".jpg", f"_aug{i}.jpg")

                cv2.imwrite(os.path.join(output_dir, "images", split, aug_name), aug_img)
                with open(os.path.join(output_dir, "labels", split, aug_name.replace(".jpg", ".txt")), "w") as f:
                    for cls, b in zip(aug_labels, aug_boxes_yolo):
                        f.write(f"{int(cls)} {' '.join(f'{v:.6f}' for v in b)}\n")
            except Exception as e:
                print(f"[!] Помилка при аугментації {fname} (i={i}): {e}")
                skipped_augs += 1

print("✅ Аугментація завершена.")
print(f"❗️ Пропущено невалідних боксів: {bad_boxes_total}")
print(f"⚠️ Пропущено аугментацій через помилки: {skipped_augs}")
print(f"📁 Результат збережено у {output_dir}/")


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:58: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


✅ Аугментація завершена.
❗️ Пропущено невалідних боксів: 0
⚠️ Пропущено аугментацій через помилки: 0
📁 Результат збережено у /kaggle/working/cats-yolo-dataset-aug/


In [6]:
# 1. Додаємо бібліотеку ultralytics до Python шляху


# 2. Імпортуємо модуль YOLO

#!pip install ultralytics
from ultralytics import YOLO


# 3. Створюємо модель (можна обрати n/s/m/l/x)
model = YOLO("yolo11m.yaml") 

# 4. Навчання моделі
model.train(
    data="/kaggle/input/cats-yolo-v2/cats-yolo-v2/data.yaml",
    epochs=150,
    imgsz=640,
    batch=13,
    name="yolo11_cat_detector",
    device=0,
    project="/kaggle/working/yolo_results"
)


Ultralytics 8.3.131 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=13, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/cats-yolo-v2/cats-yolo-v2/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11_cat_detector4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.

train: Scanning /kaggle/input/cats-yolo-v2/cats-yolo-v2/train/labels... 401 images, 0 backgrounds, 0 corrupt: 100%|██████████| 401/401 [00:00<00:00, 944.05it/s]

WARNING ⚠️ train: Cache directory /kaggle/input/cats-yolo-v2/cats-yolo-v2/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 162.0±72.9 MB/s, size: 1770.2 KB)


val: Scanning /kaggle/input/cats-yolo-v2/cats-yolo-v2/valid/labels... 85 images, 0 backgrounds, 0 corrupt: 100%|██████████| 85/85 [00:00<00:00, 254.09it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.17it/s]


                   all         85         85       0.85      0.835      0.875      0.477
          munchkin_cat         35         35      0.928       0.74      0.877      0.471
         scottish_fold         20         20       0.81        0.9       0.85      0.384
          siberian_cat         30         30      0.812      0.865      0.898      0.576


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 11.2ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to /kaggle/working/yolo_results/yolo11_cat_detector4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b3a340ddb90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [27]:
!pwd
!cd yolo_results
!pwd


/kaggle/working
/bin/bash: line 1: cd: yolo_result: No such file or directory
/kaggle/working


In [11]:
# !pip install -q ultralytics
import os
import cv2
from ultralytics import YOLO

# 1. Завантаження моделі
model = YOLO('/kaggle/working/yolo_results/yolo11_cat_detector4/weights/best.pt')  # заміни на свій шлях

# 2. Директорії для вводу/виводу
input_dir = '/kaggle/input/media-samples/media-samples'  # директорія з відео
output_dir = '/kaggle/working/output_media'
os.makedirs(output_dir, exist_ok=True)

# 3. Підтримувані формати
video_exts = {'.mp4', '.avi', '.mov', '.mkv', '.webm'}

# 4. Обробка відеофайлів
for fname in os.listdir(input_dir):
    input_path = os.path.join(input_dir, fname)
    name, ext = os.path.splitext(fname.lower())

    if ext in video_exts:
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            print(f"[!] Неможливо відкрити відео {fname}")
            continue

        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        out_path = os.path.join(output_dir, f"{name}_detected.mp4")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

        frames_processed = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            results = model(frame, verbose=False)
            annotated_frame = results[0].plot()
            out.write(annotated_frame)
            frames_processed += 1

        cap.release()
        out.release()
        print(f"🎞️ Оброблено відео: {fname}, кадрів: {frames_processed}")

print("✅ Всі відеофайли оброблені. Збережено в 'output_media/'")


🎞️ Оброблено відео: scottish_fold2.mp4, кадрів: 702
🎞️ Оброблено відео: scottish_fold1.mp4, кадрів: 245
🎞️ Оброблено відео: siberian_cat1.mp4, кадрів: 369
🎞️ Оброблено відео: cat.mp4, кадрів: 481
🎞️ Оброблено відео: siberian_cat.mp4, кадрів: 333
✅ Всі відеофайли оброблені. Збережено в 'output_media/'
